In [1]:
from google.colab import drive
drive.mount('/content/drive')
# カレントディレクトリの指定
import os
os.chdir('/content/drive/MyDrive/分析コンペ/05_ProbSpace/民泊サービスの宿泊料金予測/')

Mounted at /content/drive


In [2]:
! nvidia-smi

Sat Mar 19 05:11:46 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [3]:
import random
import os
import datetime
import numpy as np
import pandas as pd
import warnings
import gc
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline

from src.config import *
import src.preprocessing as pr

warnings.filterwarnings('ignore')

In [25]:
df_train = pd.read_csv('input/train_data.csv', parse_dates=[COL_LAST_REVIEW], dtype=DICT_DTYPES)
df_test = pd.read_csv('input/test_data.csv', parse_dates=[COL_LAST_REVIEW], dtype=DICT_DTYPES)
df_train_station_info = pd.read_csv('input/train_data_distance_from_station.csv', dtype=DICT_DTYPES)
df_test_station_info = pd.read_csv('input/test_data_distance_from_station.csv', dtype=DICT_DTYPES)
df_train_svd_name = pd.read_csv('input/train_data_name_features.csv')
df_test_svd_name = pd.read_csv('input/test_data_name_features.csv')
df_train_rf_stacking = pd.read_csv('input/rf_out_of_fold.csv')
df_test_rf_stacking = pd.read_csv('submit/submission_rf_1-0-0.csv')
sample_sub = pd.read_csv('input/submission.csv')

In [6]:
# HyperParameters
DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')
EPOCHS = 70
BATCH_SIZE = 64
LEARNING_RATE = 1e-3
WEIGHT_DECAY = 1e-5
NFOLDS = 5
EARLY_STOPPING_STEPS = 11
EARLY_STOP = True

LIST_USE_COL = [COL_NEIGHBOURHOOD,
                COL_LATITUDE,
                COL_LONGITUDE,
                COL_ROOM_TYPE,
                COL_MINIMUM_NIGHTS,
                COL_NUMBER_OF_REVIEWS,
                COL_ELAPSED_DAYS,
                COL_REVIEWS_PER_MONTH,
                COL_AVAILABILITY_365]

LIST_LABEL_ENC = [COL_NEIGHBOURHOOD,
                  COL_ROOM_TYPE]

In [7]:
class TrainDataset(Dataset):
    def __init__(self, features, target):
        super().__init__()
        self.features = features.reset_index(drop=True)
        self.target = target.reset_index(drop=True)

    def __len__(self):
        return self.features.shape[0]

    def __getitem__(self, idx):
        x = torch.tensor(self.features.loc[idx, :].values, dtype=torch.float)
        y = torch.tensor(self.target[idx], dtype=torch.float)
        return x, y


class TestDataset(Dataset):
    def __init__(self, features):
        super().__init__()
        self.features = features.reset_index(drop=True)
        
    def __len__(self):
        return self.features.shape[0]

    def __getitem__(self, idx):
        x = torch.tensor(self.features.loc[idx, :].values, dtype=torch.float)
        return x


class Model(nn.Module):
    def __init__(self, num_features, num_targets, hidden_size_1, hidden_size_2):
        super(Model, self).__init__()
        self.batch_norm1 = nn.BatchNorm1d(num_features)
        self.dropout1 = nn.Dropout(0.2)
        self.dense1 = nn.utils.weight_norm(nn.Linear(num_features, hidden_size_1))
        
        self.batch_norm2 = nn.BatchNorm1d(hidden_size_1)
        self.dropout2 = nn.Dropout(0.2)
        self.dense2 = nn.utils.weight_norm(nn.Linear(hidden_size_1, hidden_size_2))

        self.batch_norm3 = nn.BatchNorm1d(hidden_size_2)
        self.dropout3 = nn.Dropout(0.2)
        self.dense3 = nn.utils.weight_norm(nn.Linear(hidden_size_2, hidden_size_2))
        
        self.batch_norm4 = nn.BatchNorm1d(hidden_size_2)
        self.dropout4 = nn.Dropout(0.25)
        self.dense4 = nn.utils.weight_norm(nn.Linear(hidden_size_2, num_targets))
    
    def forward(self, x):
        x = self.batch_norm1(x)
        x = self.dropout1(x)
        x = F.relu(self.dense1(x))
        
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        x = F.relu(self.dense2(x))

        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = F.relu(self.dense3(x))
        
        x = self.batch_norm4(x)
        x = self.dropout4(x)
        x = self.dense4(x)
        
        return x

In [8]:
def train_fn(train_loader, model, criterion, optimizer, device):
    model.train()
    train_loss = 0.0
    train_rmsle = 0.0

    for features, labels in tqdm(train_loader):
        optimizer.zero_grad()  # 勾配情報を初期化
        features = features.to(device)  # GPU上に転送
        labels = labels.to(device)  # GPU上に転送
        outputs = model(features)  # GPU上に転送
        outputs = torch.squeeze(outputs)  # 1次元に変換
        loss = torch.sqrt(criterion(outputs, labels))  # 損失を計算
        preds = outputs.detach().cpu().numpy()  # numpyの配列に変換
        labels = labels.detach().cpu().numpy()  # numpyの配列に変換

        train_loss += loss.item()  # 値だけ取得
        train_rmsle += np.sqrt(mean_squared_error(labels, preds))  # 評価指標の計算
        loss.backward()  # 勾配を計算
        optimizer.step()  # 勾配を更新

    train_loss /= len(train_loader)  # 1epoch分の平均を計算
    train_rmsle /= len(train_loader)  # 1epoch分の平均を計算

    return train_loss, train_rmsle


def valid_fn(valid_loader, model, criterion, device):
    model.eval()
    valid_loss = 0.0
    valid_rmsle = 0.0
    valid_preds = []

    for features, labels in tqdm(valid_loader):
        features = features.to(device)
        labels = labels.to(device)
        with torch.no_grad():
            outputs = model(features)
            outputs = torch.squeeze(outputs)
            loss = torch.sqrt(criterion(outputs, labels))
            preds = outputs.detach().cpu().numpy()
            labels = labels.detach().cpu().numpy()

        valid_loss += loss.item()
        valid_rmsle += np.sqrt(mean_squared_error(labels, preds))
        valid_preds.append(preds)

    valid_loss /= len(valid_loader)
    valid_rmsle /= len(valid_loader)
    valid_preds = np.concatenate(valid_preds)

    return valid_loss, valid_rmsle, valid_preds


def inference_fn(model, dataloader, device):
    model.eval()
    preds = []
    
    for features in dataloader:
        features = features.to(device)

        with torch.no_grad():
            outputs = model(features)
        
        preds.append(outputs.detach().cpu().numpy())
        
    preds = np.concatenate(preds)
    
    return preds

In [9]:
def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

seed_everything(seed=42)

In [10]:
df_all = pd.concat([
    df_train,
    df_test
]).reset_index(drop=True)

In [11]:
# 2020.4.30からの経過日数を特徴量として追加
df_all[COL_ELAPSED_DAYS] = (datetime.datetime(2020, 4, 30) - df_all[COL_LAST_REVIEW]).dt.days

In [12]:
df_all.fillna(0, inplace=True)

In [13]:
df_all = df_all[LIST_USE_COL]

In [14]:
df_all = pd.get_dummies(df_all, columns=LIST_LABEL_ENC, drop_first=True)

In [15]:
df_all.head()

,latitude,longitude,minimum_nights,number_of_reviews,elapsed_days,reviews_per_month,availability_365,neighbourhood_Arakawa Ku,neighbourhood_Bunkyo Ku,neighbourhood_Chiyoda Ku,...,neighbourhood_Shibuya Ku,neighbourhood_Shinagawa Ku,neighbourhood_Shinjuku Ku,neighbourhood_Suginami Ku,neighbourhood_Sumida Ku,neighbourhood_Taito Ku,neighbourhood_Toshima Ku,room_type_Hotel room,room_type_Private room,room_type_Shared room
0,35.68185,139.80310,1,55,5.0,2.21,173,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,35.72063,139.78536,6,72,36.0,2.11,9,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,35.74723,139.82349,1,18,38.0,3.46,288,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,35.68456,139.68077,1,2,28.0,1.76,87,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,35.69840,139.70467,1,86,91.0,2.00,156,0,0,0,...,0,0,1,0,0,0,0,0,0,0


In [16]:
df_all_station_info = pd.concat([df_train_station_info, df_test_station_info], axis=0).reset_index(drop=True)
pipe = Pipeline([
    ('scale', StandardScaler()),
    ('pca', PCA(n_components=10, random_state=0))
])

features_tmp = pipe.fit_transform(df_all_station_info)
df_distance_features = pd.DataFrame(features_tmp, columns=[f'PCA_{i+1}' for i in range(features_tmp.shape[1])])

In [17]:
df_train_distance_features = df_distance_features[:df_train.shape[0]].reset_index(drop=True)
df_test_distance_features = df_distance_features[df_train.shape[0]:].reset_index(drop=True)

In [21]:
df_train_rf_stacking.head()

,rf_stacking
0,9.744429
1,9.748376
2,9.691751
3,9.030463
4,9.734504


In [26]:
df_test_rf_stacking['rf_stacking'] = np.log1p(df_test_rf_stacking[COL_Y])
df_test_rf_stacking = df_test_rf_stacking[['rf_stacking']]
df_test_rf_stacking.head()

,rf_stacking
0,9.502303
1,9.625121
2,9.619025
3,9.467607
4,9.486467


In [27]:
X = df_all[:df_train.shape[0]].reset_index(drop=True)
X = pd.concat([X, df_train_distance_features, df_train_svd_name, df_train_rf_stacking], axis=1)
y = np.log1p(df_train[COL_Y])

X_inference = df_all[df_train.shape[0]:].reset_index(drop=True)
X_inference = pd.concat([X_inference, df_test_distance_features, df_test_svd_name, df_test_rf_stacking], axis=1)

In [28]:
scale = StandardScaler()
scale.fit(X)
X = pd.DataFrame(data=scale.transform(X), columns=X.columns)
X_inference = pd.DataFrame(data=scale.transform(X_inference), columns=X_inference.columns)

In [29]:
X.head()

,latitude,longitude,minimum_nights,number_of_reviews,elapsed_days,reviews_per_month,availability_365,neighbourhood_Arakawa Ku,neighbourhood_Bunkyo Ku,neighbourhood_Chiyoda Ku,...,name_nmf_42,name_nmf_43,name_nmf_44,name_nmf_45,name_nmf_46,name_nmf_47,name_nmf_48,name_nmf_49,name_nmf_50,rf_stacking
0,-0.452253,1.100902,-0.274144,0.715964,-0.752875,0.559342,0.136454,-0.171366,-0.110264,-0.13584,...,0.012767,-0.202456,-0.210780,-0.123024,-0.286139,-0.156764,-0.153178,-0.199542,-0.152840,0.401572
1,0.514113,0.777728,0.313812,1.134846,-0.458280,0.489338,-1.280089,-0.171366,-0.110264,-0.13584,...,-0.131782,-0.202456,-0.210780,-0.123024,-0.286139,-0.156764,-0.153178,-0.199542,-0.183797,0.407349
2,1.176963,1.472351,-0.274144,-0.195720,-0.439274,1.434393,1.129761,-0.171366,-0.110264,-0.13584,...,-0.211190,-0.202456,-0.210780,-0.123024,-0.286139,-0.156764,-0.153178,-0.199542,-0.183797,0.324463
3,-0.384722,-1.127610,-0.274144,-0.589961,-0.534304,0.244324,-0.606367,-0.171366,-0.110264,-0.13584,...,-0.211190,-0.202456,-0.210780,-0.123024,-0.286139,-0.156764,-0.153178,-0.199542,-0.183797,-0.643508
4,-0.039840,-0.692218,-0.274144,1.479807,0.064390,0.412334,-0.010383,-0.171366,-0.110264,-0.13584,...,-0.211190,-0.202456,7.127962,-0.123024,-0.286139,-0.156764,-0.153178,-0.199542,-0.183797,0.387044


In [30]:
num_features= X.shape[1]
num_targets= 1
hidden_size_1=512
hidden_size_2=256

In [31]:
kf = KFold(n_splits=NFOLDS, shuffle=True, random_state=0)
preds_oof = np.zeros(X.shape[0])
for i, (train_idx, valid_idx) in enumerate(kf.split(X)):
    model = Model(num_features, num_targets, hidden_size_1, hidden_size_2)
    model.to(DEVICE)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(params=model.parameters(), lr=0.001)
    print(f'fold: {i}')
    print('='*50)
    X_train, X_valid = X.loc[train_idx, :], X.loc[valid_idx, :]
    y_train, y_valid = y.loc[train_idx], y.loc[valid_idx]
    train_dataset = TrainDataset(X_train, y_train)
    valid_dataset = TrainDataset(X_valid, y_valid)
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, num_workers=4, shuffle=True)
    valid_loader = DataLoader(valid_dataset, batch_size=BATCH_SIZE, num_workers=4, shuffle=False)

    for epoch in range(EPOCHS):
        train_loss, train_rmsle = train_fn(train_loader, model, criterion, optimizer, DEVICE)
        valid_loss, valid_rmsle, valid_preds = valid_fn(valid_loader, model, criterion, DEVICE)
        print(f'train_loss: {train_loss:.4f}, train_rmsle: {train_rmsle:.4f}')
        print(f'valid_loss: {valid_loss:.4f}, valid_rmsle: {valid_rmsle:.4f}')
    
    print('='*50)
    print(f'save model: models/nn_fold{i + 1}_.pth')
    print('='*50)
    torch.save(model.state_dict(), f"models/nn_fold{i + 1}_.pth")

    del model, criterion, optimizer
    gc.collect()

fold: 0


100%|██████████| 32/32 [00:00<00:00, 98.65it/s]


train_loss: 8.7888, train_rmsle: 8.7888
valid_loss: 7.8182, valid_rmsle: 7.8182


100%|██████████| 32/32 [00:00<00:00, 99.47it/s] 


train_loss: 6.5716, train_rmsle: 6.5716
valid_loss: 5.1588, valid_rmsle: 5.1588


...

train_loss: 0.4455, train_rmsle: 0.4455
valid_loss: 0.5175, valid_rmsle: 0.5175


100%|██████████| 32/32 [00:00<00:00, 93.19it/s]


train_loss: 0.4555, train_rmsle: 0.4555
valid_loss: 0.5082, valid_rmsle: 0.5082


100%|██████████| 32/32 [00:00<00:00, 84.54it/s]


train_loss: 0.4482, train_rmsle: 0.4482
valid_loss: 0.5187, valid_rmsle: 0.5187
save model: models/nn_fold5_.pth


In [33]:
list_preds_tmp = []
inference_dataset = TestDataset(X_inference)
inference_loader = DataLoader(inference_dataset, batch_size=BATCH_SIZE, shuffle=False)
for i in range(1, 6):
    model = Model(num_features, num_targets, hidden_size_1, hidden_size_2)
    model.load_state_dict(torch.load(f"models/nn_fold{i}_.pth"))
    model.to(DEVICE)
    predictions = inference_fn(model, inference_loader, DEVICE)
    list_preds_tmp.append(predictions)

In [34]:
df_preds = pd.DataFrame({'model_1': np.squeeze(list_preds_tmp[0]),
                         'model_2': np.squeeze(list_preds_tmp[1]),
                         'model_3': np.squeeze(list_preds_tmp[2]),
                         'model_4': np.squeeze(list_preds_tmp[3]),
                         'model_5': np.squeeze(list_preds_tmp[4])})

df_preds['pred_avg'] = df_preds.mean(axis=1)

In [35]:
df_preds.head()

,model_1,model_2,model_3,model_4,model_5,pred_avg
0,9.258580,9.501192,9.291500,9.368126,9.427443,9.369368
1,10.893875,10.375762,10.796649,11.289084,11.036330,10.878341
2,9.175266,9.468945,9.361102,9.037961,9.428020,9.294259
3,9.192974,9.210188,9.764295,9.464609,8.785627,9.283539
4,8.781078,9.237726,8.877485,8.834661,9.290231,9.004236


In [36]:
sample_sub[COL_Y] = np.expm1(df_preds['pred_avg'])

In [37]:
sample_sub.head()

,id,y
0,1,11722.699219
1,2,53014.558594
2,3,10874.404297
3,4,10758.440430
4,5,8136.483398


In [38]:
sample_sub.to_csv('submit/submission_neural_network_fold_avg.csv', index=False)